In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Pre-processing and file reading (DO NOT RUN THIS GLOVE PART)

In [ ]:
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/')

from pickle_vocab import main as pickle_vocab
from cooc import main as cooc
from glove_solution import main as glove


In [ ]:
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/unique/short/')

In [ ]:
%%bash

chmod u+x ./build_vocab.sh
./build_vocab.sh
chmod u+x ./cut_vocab.sh
./cut_vocab.sh

In [ ]:
pickle_vocab()
cooc()

# GloVe 

In [ ]:
glove()

In [ ]:
X = np.load('embeddings.npy')
print(X.shape)

In [ ]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

# START FROM HERE : Application of Doc2Vec to dataset (OUR)

In [65]:
import os
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/unique/short')


# Doc2Vec by itself (build its vocab with Word2Vec)

Building the Doc2Vec model following the example: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

In [34]:
import gensim
import os
import collections
import smart_open
import random

import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"


In [35]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [36]:
train_corpus = list(read_corpus('all_short.txt'))

In [37]:
#setting hyperparameters
vector_size = 25
window_size = 10
min_count = 5
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 0 #0 = dbow; 1 = dmpv


model = gensim.models.doc2vec.Doc2Vec(size=vector_size, min_count=min_count, sample=sampling_threshold, workers=cores, hs=0,dm=dm,negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)

In [38]:
model.build_vocab(train_corpus)


One important thing to note is that you can now infer a vector for any piece of text without having to re-train the model by passing a list of words to the model.infer_vector function.

In [42]:
# infearing the positive vectors 
fname = 'train_pos_unique.txt'
num_lines = sum(1 for line in open(fname))
X_pos = np.zeros((num_lines, vector_size))

# define a function that create a list of lists of words: we need to infear to doc2vec model a list of separate word
# to obtain the corresponding vectors from the entire twit

def read_words(words_file):
    row = []
    for line in open(fname, 'r', encoding="utf-8-sig"):
        word = [w for w in line.split()]
        row.append(word)
        
    return row
    
    
content_pos = read_words(fname)

for i in range(len(content_pos)):
    X_pos[i,:] = model.infer_vector(content_pos[i])

X_pos = np.array(X_pos)

In [46]:
fname = 'train_neg_unique.txt'
num_lines = sum(1 for line in open(fname))
X_neg = np.zeros((num_lines, vector_size))

content_neg = read_words(fname)

for i in range(len(content_neg)):
    X_neg[i,:] = model.infer_vector(content_neg[i])

X_neg = np.array(X_neg)

In [47]:
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [48]:
print(X_pos.shape)
print(X_neg.shape)

(90233, 25)
(91088, 25)


In [49]:
X_pos_neg = np.concatenate([X_pos, X_neg])
y_pos_neg = np.concatenate([y_pos, y_neg])

In [61]:
ind = np.arange(X_pos_neg.shape[0])
ind_shuf = np.random.permutation(ind)

X_pos_neg = X_pos_neg[ind_shuf, :]
y_pos_neg = y_pos_neg[ind_shuf]

# SVM

In [62]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_pos_neg, y_pos_neg) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Submission

In [66]:
# Reading the ids from the file
ids_test = np.zeros(10000)

with open('test_data_unique.txt', 'r', encoding="utf-8-sig") as f:
    i = 0
    for line in f:
        id_ = line.lstrip().split(',')[0]
        ids_test[i] = id_
        i = i+1
        
        


# Building the X matrix
fname = 'test_data_unique_no_id.txt'
num_lines = sum(1 for line in open(fname))
X_test = np.zeros((num_lines, vector_size))

content = read_words(fname)

for i in range(len(content)):
    X_test[i,:] = model.infer_vector(content[i])


In [67]:
y_pred = clf.predict(X_test)


In [68]:
X_test.shape

(10000, 25)

In [71]:
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/COMMON/')

In [72]:
from create_csv_submission import create_csv_submission

name = 'subB'
create_csv_submission(ids_test, y_pred, name)

In [73]:
X_test = np.array(X_test)

In [74]:
X_test.shape

(10000, 25)